# Neural Machine Translation - Assignment 2

In this task, a neural machine translation (NMT) system is developed to translate text from one language into another. For this, italian data is chosen to train the models, and data processing is performed and sequence2sequence neural model with attention and without attention is trained and tested. BLEU score is used evaluation metric.


## Section 1- Data Collection and Preprocessing 


---


**Task 1**

---

There are few datasets to train an NMT system available from Tatoeba Project (http://www.manythings.org/anki/) or OPUS project (http://opus.nlpl.eu/).

*  Download a language pair (preferably European language) and **extract** the file(s) and upload it to CoLab
*  Create a list of lines by splitting the text file at every occurrence accordingly, i.e. source and target language and remove copyright information
*  Print number of sentences
*  Limit the number of sentences to 10,000 lines (but more than 5,000 lines)
*  Split the data into train and test [You can split validation set here or while training use Keras validation_split option]
*  Print 100th sentence in original script[ not unicode] for source and target language

###### Imports

In [1]:
#imports
import os, sys
import string,re
import numpy as np
import unicodedata
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
import keras.backend as K

Using TensorFlow backend.


###### Read 10000 lines of the file ita.txt

In [2]:
#initialize
BATCH_SIZE = 64  # Batch size used for training.
EPOCHS = 20  # Number of epochs for training.
LATENT_DIM = 256  # Latent dimension of LSTM model.
NUM_SAMPLES = 10000  # Number sentences to extract from file.
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100 # number of dimensions used in the embedding(glove embeddings) 

file = open("ita.txt", encoding='utf-8')
count = 0
sample = []
lines = []
for line in file:
    if '\t' not in line:
        continue
    input_text, translation, *rest = line.rstrip().split('\t')
    input_text, translation = input_text.strip(), translation.strip()
    if(len(input_text.split()) > 4 and input_text not in sample):
        lines.append([input_text,translation])
        sample.append(input_text)
        count+=1
    if count == NUM_SAMPLES :
        break


In [3]:
print(len(lines))
print(lines[100])

10000
['I want to eat it.', 'Voglio mangiarlo.']


**Task 2** 

---

* Add '<bof\>' to denote beginning of sentence and '<eos\>' to denote the end of the sentence to each target line.
* Preprocess (word tokenisation, lowercasing) the text.

###### Pre processing the lines of the file in the target and source language.<br>
Add '<bof\>' to the beginning of the line in the target and add '<eos\>' to the end of the line in the target. 

In [4]:
#pre processing
input_texts = [] # sentence line in source language
target_texts = [] # sentence line in target language
target_texts_inputs = [] # sentence line will be used as input text for decoder model

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# Reference:- Tensorflow nmt preprocessing
def preprocess_sentence(line):
    line = unicode_to_ascii(line.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    line = re.sub(r"([?.!,])", r" \1 ", line)
    # replace extra spaces from a sentence to a single space
    line = re.sub(r'[" "]+', " ", line) 
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    line = re.sub(r"[^a-zA-Z?.!,]+", " ", line)
    line = line.strip()
    return line

for line in lines:
    input_text,translation = line
    input_text,translation = preprocess_sentence(input_text), preprocess_sentence(translation)
    target_text = translation + ' <eos>'
    target_text_input = '<bof> ' + translation
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_inputs.append(target_text_input)


In [5]:
source_vocab = []
target_vocab = []

len1 = 0 
len2 = 0
for input_line in input_texts:
    if(len(input_line.split())>len1):
        len1 = len(input_line.split())
    for word in input_line.split():
        source_vocab.append(word)
        
for target_line in target_texts:
    if(len(target_line.split())>len2):
        len2 = len(input_line.split())
    for word in target_line.split():
        target_vocab.append(word)
        
source_vocab, target_vocab = list(set(source_vocab)), list(set(target_vocab))


In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique source language tokens:', len(source_vocab))
print('Number of unique target language tokens:', len(target_vocab))
print('Max sequence length of source language:', len1)
print('Max sequence length of target language:', len2)
print("Source Vocabulary",source_vocab)
print("Target Vocabulary",target_vocab)

Number of samples: 10000
Number of unique source language tokens: 2355
Number of unique target language tokens: 3842
Max sequence length of source language: 9
Max sequence length of target language: 6
Source Vocabulary ['front', 'hardly', 'popular', 'bloomer', 'tiger', 'figured', 'yesterday', 'mary', 'lift', 'station', 'passed', 'opener', 'pass', 'rainy', 'careful', 'strong', 'bossy', 'zoo', 'steam', 'photo', 'upside', 'cleaned', 'tub', 'young', 'playing', 'moron', 'please', 'rely', 'useless', 'likable', 'struck', 'wealth', 'bronze', 'fat', 'getting', 'weight', 'lake', 'rowboat', 'tactful', 'ruin', 'store', 'murder', 'terrible', 'favor', 'camp', 'elegant', 'kendo', 'pigged', 'waiter', 'pen', 'cards', 'cannot', 'survive', 'sixty', 'scream', 'buy', 'pianist', 'glad', 'call', 'lean', 'sister', 'city', 'less', 'employed', 'issue', 'tomorrow', 'wood', 'bite', 'massage', 'hand', 'plus', 'ghost', 'social', 'bed', 'almost', 'by', 'pony', 'roads', 'somehow', 'pray', 'curious', 'pack', 'neat', '

**Task 3** 

---

---

*  Assign each unique word an integer value.
*  Create word embedding for your vocabulary using pre-trained Glove embeddings (http://nlp.stanford.edu/data/glove.6B.zip)
* Print the first line of the embeddings (see below) 

This section of the code tokenizes the source texts and target texts.<br>
The code below uses Tokenizer function from Keras library. This section creates a unique index for all words in the source and target texts. The a sequence of unique indexes is created for all the texts in the source and target language.<br>
Next the data is divided into training and test set. 80% of the data is used for training while 20% of the data will be used for testing the model.

In [7]:
# tokenize the inputs
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

word2idx_inputs = tokenizer_inputs.word_index

max_len_input = max(len(s) for s in input_sequences)

tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) # inefficient, oh well
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)

word2idx_outputs = tokenizer_outputs.word_index

num_words_output = len(word2idx_outputs) + 1

max_len_target = max(len(s) for s in target_sequences)

encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
encoder_inputs, encoder_tests = encoder_inputs[:int(0.8*len(encoder_inputs))], encoder_inputs[int(0.8*len(encoder_inputs)):]

decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
decoder_inputs, decoder_tests = decoder_inputs[:int(0.8*len(decoder_inputs))], decoder_inputs[int(0.8*len(decoder_inputs)):]

decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')
decoder_targets, decoder_target_tests = decoder_targets[:int(0.8*len(decoder_targets))], decoder_targets[int(0.8*len(decoder_targets)):]

This section of the code below reads glove embeddings of 100 dimension for each word to create word vectors for the source language i.e. English from the pre-trained embeddings file. A corresponding embedding matrix hashmap is created with the unique indexes for each word as the key with the embedding as value. The embeddings of words which is not present in the glove file is taken to be 0. This embedding matrix will be used as input for the encoder model

In [8]:
word2vec = {}
with open(r'glove.6B/glove.6B.100d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    if i < MAX_NUM_WORDS:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all zeros.
          embedding_matrix[i] = embedding_vector

In [9]:
# print first line of embeddings here)
first_line = encoder_inputs[0]
print("First Line: ",input_texts[0])
print("Word to index representation of first line: ", first_line)

print("Glove embeddings of first line")
for index in first_line:
    print(embedding_matrix[index])

First Line:  i am a shy boy .
Word to index representation of first line:  [  0   0   0   1  53   3 293 176]
Glove embeddings of first line
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

## Section 2 Translation Model training



---



**Task 4**
* Provide code for the encoder using Keras LSTM
* Provide code for the decoder using Keras LSTM
* Train the sequence2sequence (encoder-decoder) model


The code is the section below is used for creating embedding layer which will be used for providing input to the encoder.

In [10]:
# create embedding layer
embedding_layer = Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_len_input)

decoder_targets_one_hot = np.zeros((int(0.8*len(input_texts)), max_len_target, num_words_output), dtype='float32' )

for i, d in enumerate(decoder_targets):
    for t, word in enumerate(d):
        if word != 0:
            decoder_targets_one_hot[i, t, word] = 1

The code in the section below is used for creating encoder and decoder.

In [11]:
# encoder code goes here
encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(
  LATENT_DIM,
  return_state=True,
)
encoder_outputs, h, c = encoder(x)

encoder_states = [h, c]









Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# decoder code goes here
decoder_inputs_placeholder = Input(shape=(max_len_target,))

decoder_embedding = Embedding(num_words_output, EMBEDDING_DIM)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(
  LATENT_DIM,
  return_sequences=True,
  return_state=True,
)
decoder_outputs, _, _ = decoder_lstm(
  decoder_inputs_x,
  initial_state=encoder_states
)

decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Defining the model and printing a summary of the layers in the model

In [13]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 8, 100)       235200      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 12, 100)      384400      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (

Training the model on the data and saving the trained model

In [14]:
model.fit(
  [encoder_inputs, decoder_inputs], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=0.1,
) 
model.save('seq2seq_source_target.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 7200 samples, validate on 800 samples
Epoch 1/20
7200/7200 [==============================] - 37s 5ms/step - loss: 2.4981 - acc: 0.1469 - val_loss: 2.2746 - val_acc: 0.2057
Epoch 2/20
7200/7200 [==============================] - 34s 5ms/step - loss: 2.0065 - acc: 0.2170 - val_loss: 2.0151 - val_acc: 0.2525
Epoch 3/20
7200/7200 [==============================] - 35s 5ms/step - loss: 1.7184 - acc: 0.2510 - val_loss: 1.8283 - val_acc: 0.2756
Epoch 4/20
7200/7200 [==============================] - 35s 5ms/step - loss: 1.5106 - acc: 0.2798 - val_loss: 1.6437 - val_acc: 0.2975
Epoch 5/20
7200/7200 [==============================] - 35s 5ms/step - loss: 1.3536 - acc: 0.3007 - val_loss: 1.5672 - val_acc: 0.3128
Epoch 6/20
7200/7200 [==============================] - 35s 5ms/

C:\Users\grohi\Anaconda3\lib\site-packages\keras\engine\topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


## Section 3 Testing

---

**Task 5**

* Use the trained model to translate the text from the source into the target language. 
* Use the test/evaluation set (see Section 1) and perform an automatic evaluation with the BLEU metric. 
You can use the NLTK library to calculate BLEU.

This section uses the encoder and decoder model weights to create a model which will be used for for prediction.<br>
A fucntion decode_sequence() is defined which takes a input seq of indexes for the words. This function predicts words for the translated language. 

In [15]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, h, c = decoder_lstm(
  decoder_inputs_single_x,
  initial_state=decoder_states_inputs
)
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs_single] + decoder_states_inputs, 
  [decoder_outputs] + decoder_states
)

idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<bof>']
    eos = word2idx_outputs['<eos>']

    # Create the translation
    output_sentence = []
    for _ in range(max_len_target):
        output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

        # Get the next word
        idx = np.argmax(output_tokens[0, 0, :])

        # check if end of sentence tag is generated for break
        if eos == idx:
          break

        word = ''
        if idx > 0:
          word = idx2word_trans[idx]
          output_sentence.append(word)

        target_seq[0, 0] = idx

        states_value = [h, c]

    return ' '.join(output_sentence)

BLEU score is calculated for the test data. Then a average bleu score is calculated. <br>
Average BLEU score for Seq2Seq model without attentions is **0.391**

In [16]:
#BLEU Score Calculation
avg = 0
from nltk import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothing = SmoothingFunction()

for i in range(len(encoder_tests)):
    translation = decode_sequence(encoder_tests[i:i+1])
    hyp = translation.split()
    target_text = target_texts[int(0.8*len(input_texts))+i].split()
    ref = target_text[:len(target_text)-1]
    BLEUscore = bleu([ref], hyp, smoothing_function=smoothing.method2)
    #print(hyp,ref,BLEUscore)
    avg+=BLEUscore
print("Average BLEU Score for Seq2Seq model without attention: ",avg/(i+1))

Average BLEU Score for Seq2Seq model without attention:  0.39165789159749353


# Section 4 Attention

---



**Task 5** <br><br>
Sequence2Sequence

* Extend the existing Seq2Seq model with an attention mechanism
* Create sequence2sequence model with attention 
* Train the model with the same data from Section 1 
* Translate the evaluation set using the sequence2sequence attention model 
* Evaluate the translations made with the sequence2sequence attention model and compare it with the model without attention using BLEU 

Seq2Seq encoder decoder model with attention is coded. The same source language texts will be used for the new model. Instead of a regualar LSTM, Bidirection LSTM will be used for the encoder model. Attention is performed this model which improves the bleu score. The bleu score for this model after attenntion is **0.421**

In [17]:
def softmax_over_time(x):
    assert(K.ndim(x) > 2)
    e = K.exp(x - K.max(x, axis=1, keepdims=True))
    s = K.sum(e, axis=1, keepdims=True)
    return e / s

LATENT_DIM = 400 # Latent dimension of LSTM model for encoder.
LATENT_DIM_DECODER = 400 # Latent dimension of LSTM model for decoder.
EPOCHS = 30  # Number of epochs for training.

encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = Bidirectional(LSTM(
  LATENT_DIM,
  return_sequences=True,
  # dropout=0.5 
))
encoder_outputs = encoder(x)

decoder_inputs_placeholder = Input(shape=(max_len_target,))

decoder_embedding = Embedding(num_words_output, EMBEDDING_DIM)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

# Attention 
attn_repeat_layer = RepeatVector(max_len_input)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10, activation='tanh')
attn_dense2 = Dense(1, activation=softmax_over_time)
attn_dot = Dot(axes=1) 

def one_step_attention(h, st_1):
    st_1 = attn_repeat_layer(st_1)
    x = attn_concat_layer([h, st_1])
    x = attn_dense1(x)
    alphas = attn_dense2(x)
    context = attn_dot([alphas, h])
    return context


# the decoder (after attention)
decoder_lstm = LSTM(LATENT_DIM_DECODER, return_state=True)
decoder_dense = Dense(num_words_output, activation='softmax')

initial_s = Input(shape=(LATENT_DIM_DECODER,), name='s0')
initial_c = Input(shape=(LATENT_DIM_DECODER,), name='c0')
context_last_word_concat_layer = Concatenate(axis=2)

s = initial_s
c = initial_c

outputs = []
for t in range(max_len_target): 
  
    context = one_step_attention(encoder_outputs, s)
    selector = Lambda(lambda x: x[:, t:t+1])
    xt = selector(decoder_inputs_x)

    decoder_lstm_input = context_last_word_concat_layer([context, xt])
    o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])

    decoder_outputs = decoder_dense(o)
    outputs.append(decoder_outputs)



def stack_and_transpose(x):
    x = K.stack(x)
    x = K.permute_dimensions(x, pattern=(1, 0, 2)) 
    return x

stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)

# create the model
model = Model(
  inputs=[
    encoder_inputs_placeholder,
    decoder_inputs_placeholder,
    initial_s, 
    initial_c,
  ],
  outputs=outputs
)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# training the model
z = np.zeros((len(encoder_inputs), LATENT_DIM_DECODER)) # initial [s, c]
r = model.fit(
  [encoder_inputs, decoder_inputs, z, z], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=0.1
)
# model.save('seq2seq_source_target_with_attention.h5')

Train on 7200 samples, validate on 800 samples
Epoch 1/30
7200/7200 [==============================] - 106s 15ms/step - loss: 2.4522 - acc: 0.1490 - val_loss: 2.1789 - val_acc: 0.2287
Epoch 2/30
7200/7200 [==============================] - 99s 14ms/step - loss: 1.8100 - acc: 0.2427 - val_loss: 1.8106 - val_acc: 0.2791
Epoch 3/30
7200/7200 [==============================] - 96s 13ms/step - loss: 1.4635 - acc: 0.2885 - val_loss: 1.5673 - val_acc: 0.3098
Epoch 4/30
7200/7200 [==============================] - 95s 13ms/step - loss: 1.2390 - acc: 0.3188 - val_loss: 1.4223 - val_acc: 0.3367
Epoch 5/30
7200/7200 [==============================] - 95s 13ms/step - loss: 1.0707 - acc: 0.3451 - val_loss: 1.3331 - val_acc: 0.3527
Epoch 6/30
7200/7200 [==============================] - 97s 13ms/step - loss: 0.9351 - acc: 0.3664 - val_loss: 1.2634 - val_acc: 0.3689
Epoch 7/30
7200/7200 [==============================] - 96s 13ms/step - loss: 0.8225 - acc: 0.3849 - val_loss: 1.2309 - val_acc: 0.3709


Weights from encoder decoder will be used for for prediction i.e. translation of sentences

In [19]:
encoder_model = Model(encoder_inputs_placeholder, encoder_outputs)

encoder_outputs_as_input = Input(shape=(max_len_input, LATENT_DIM * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

context = one_step_attention(encoder_outputs_as_input, initial_s)

decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])

o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)

# create the model object
decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)

idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}

def decode_sequence(input_seq):

    enc_out = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<bof>']
    eos = word2idx_outputs['<eos>']
    s = np.zeros((1, LATENT_DIM_DECODER))
    c = np.zeros((1, LATENT_DIM_DECODER))

    # Create the translation
    output_sentence = []
    for _ in range(max_len_target):
        o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        # Get next word
        idx = np.argmax(o.flatten())

        # check if end of sentence tag is generated for break
        if eos == idx:
          break

        word = ''
        if idx > 0:
          word = idx2word_trans[idx]
          output_sentence.append(word)
        target_seq[0, 0] = idx

    return ' '.join(output_sentence)

BLEU score will be calculated for the test lines. Average bleu score is then evaluated.

In [20]:
avg = 0
from nltk import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothing = SmoothingFunction()

for i in range(len(encoder_tests)):
    translation = decode_sequence(encoder_tests[i:i+1])
    hyp = translation.split()
    target_text = target_texts[int(0.8*len(input_texts))+i].split()
    ref = target_text[:len(target_text)-1]
    BLEUscore = bleu([ref], hyp, smoothing_function=smoothing.method2)
    #print(hyp,ref,BLEUscore)
    avg+=BLEUscore
print("Average BLEU Score for Seq2Seq model with attention: ",avg/(i+1))

Average BLEU Score for Seq2Seq model with attention:  0.42138376218381657
